# ☕ Análisis de Chat de WhatsApp - Edición Amistad

Este notebook procesa el archivo de texto exportado de WhatsApp para generar estadísticas sobre la amistad, actividad y vocabulario del grupo.

In [ ]:
!pip install regex demoji plotly wordcloud matplotlib pandas

In [ ]:
import pandas as pd
import re
import regex
import demoji
import numpy as np
from collections import Counter
import plotly.express as px
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import datetime

### Paso 1: Definir funciones de Parsing (Robustas)

In [ ]:
def IniciaConFechaYHora(s):
    # Patrón flexible para distintos formatos (dd/mm/yyyy o mm/dd/yyyy)
    patron = r'^(\d{1,2}/\d{1,2}/\d{2,4},? \d{1,2}:\d{2}\s?(?:[aApP]\.?[mM]\.?)?) -'
    return bool(re.match(patron, s))

def ObtenerPartes(linea):
    # Separar FechaHora del resto
    splitLinea = linea.split(' - ', 1)
    FechaHora = splitLinea[0]
    MensajeCompleto = splitLinea[1] if len(splitLinea) > 1 else ''

    # Intenta separar fecha y hora de forma robusta
    try:
        if ',' in FechaHora:
            Fecha, Hora = FechaHora.split(', ')
        else:
            Fecha, Hora = FechaHora.split(' ', 1)
    except ValueError:
        Fecha, Hora = None, None

    # Separar Autor del Mensaje (Dinámico, sin nombres hardcodeados)
    splitMensaje = MensajeCompleto.split(': ', 1)
    if len(splitMensaje) == 2:
        Miembro = splitMensaje[0]
        Mensaje = splitMensaje[1]
    else:
        Miembro = "Sistema/Notificación"
        Mensaje = MensajeCompleto

    return Fecha, Hora, Miembro, Mensaje

### Paso 2: Carga de Datos

In [ ]:
# RUTA DEL ARCHIVO (Ajustar nombre del archivo)
RutaChat = 'Data/Chat de WhatsApp con Pana Ema Bade.txt'

DatosLista = []
try:
    with open(RutaChat, encoding="utf-8") as fp:
        fp.readline() # Saltar header de cifrado
        while True:
            linea = fp.readline()
            if not linea:
                break
            linea = linea.strip()
            if IniciaConFechaYHora(linea):
                Fecha, Hora, Miembro, Mensaje = ObtenerPartes(linea)
                DatosLista.append([Fecha, Hora, Miembro, Mensaje])
            elif DatosLista:
                # Continuación de mensaje multilinea
                DatosLista[-1][-1] += " " + linea
except FileNotFoundError:
    print("⚠️ Archivo no encontrado. Verifica la ruta.")

# Crear DataFrame
df = pd.DataFrame(DatosLista, columns=['Fecha', 'Hora', 'Miembro', 'Mensaje'])

# Conversión de fechas inteligente
try:
    df['Fecha'] = pd.to_datetime(df['Fecha'], format="%d/%m/%Y")
except:
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=False)

df = df.dropna().reset_index(drop=True)

# Filtrar mensajes de sistema si se desea
df = df[df['Miembro'] != 'Sistema/Notificación']

print(f"Mensajes cargados: {df.shape[0]}")
df.head()

#### Filtrar por rango de fechas (Opcional)

In [ ]:
# Ajusta estas fechas según tu historia de amistad
start_date = '2023-01-01'
end_date = '2030-12-31'

df = df[(df['Fecha'] >= start_date) & (df['Fecha'] <= end_date)]

### Paso 3: Estadísticas Generales

In [ ]:
def ObtenerEmojis(Mensaje):
    return [c for c in regex.findall(r'\X', Mensaje) if demoji.replace(c) != c]

total_mensajes = df.shape[0]
multimedia_mensajes = df[df['Mensaje'] == '<Multimedia omitido>'].shape[0]
df['Emojis'] = df['Mensaje'].apply(ObtenerEmojis)
emojis = sum(df['Emojis'].str.len())
links = sum(df.Mensaje.apply(lambda x: len(re.findall(r'(https?://\S+)', x))))

estadistica_df = pd.DataFrame({
    'Tipo': ['Mensajes', 'Multimedia', 'Emojis', 'Links'],
    'Cantidad': [total_mensajes, multimedia_mensajes, emojis, links]
}).set_index('Tipo')

estadistica_df

#### Ranking de Emojis

In [ ]:
emojis_lista = [a for b in df.Emojis for a in b]
emoji_counts = Counter(emojis_lista).most_common(10)
emoji_df = pd.DataFrame(emoji_counts, columns=['Emoji', 'Cantidad']).set_index('Emoji')

fig = px.pie(emoji_df, values='Cantidad', names=emoji_df.index, hole=.3,
             template='plotly_dark',
             color_discrete_sequence=px.colors.sequential.Viridis) # Colores más "tech"

fig.update_traces(textposition='inside', textinfo='percent+label', textfont_size=20)
fig.update_layout(title={'text': '😎 Emojis más usados', 'x':0.5, 'xanchor': 'center'})
fig.show()

### Paso 4: Actividad de los Miembros

In [ ]:
# Miembros más activos
df_activos = df['Miembro'].value_counts().reset_index()
df_activos.columns = ['Miembro', 'Mensajes']

fig = px.bar(df_activos, x='Miembro', y='Mensajes',
             template='plotly_dark',
             color='Mensajes',
             color_continuous_scale='Teal')
fig.update_layout(title="🏆 ¿Quién habla más?")
fig.show()

### Paso 5: Análisis Temporal

In [ ]:
df['rangoHora'] = pd.to_datetime(df['Hora'], format='%H:%M').dt.hour
mensajes_hora = df.groupby('rangoHora').size().reset_index(name='Mensajes')

fig = px.line(mensajes_hora, x='rangoHora', y='Mensajes',
              color_discrete_sequence=['#00CC96'], # Color Teal Neon
              template='plotly_dark')

fig.update_layout(
    title={'text': 'Intensidad de charla por hora ⏰', 'x':0.5, 'xanchor': 'center'},
    xaxis_title='Hora (0-23)',
    font=dict(size=14))
fig.update_traces(mode='markers+lines', marker=dict(size=8))
fig.show()

In [ ]:
# Mensajes por Día de la Semana
df['DiaSemana'] = df['Fecha'].dt.day_name()
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['DiaSemana'] = pd.Categorical(df['DiaSemana'], categories=dias_orden, ordered=True)

mensajes_dia = df.groupby('DiaSemana').size().reset_index(name='Mensajes')

fig = px.bar(mensajes_dia, x='DiaSemana', y='Mensajes',
             template='plotly_dark', color_discrete_sequence=['#AB63FA'])
fig.update_layout(title="📅 Actividad por Día")
fig.show()

### Paso 6: Word Cloud (De qué hablamos)

In [ ]:
import string

# Stopwords ampliadas para chat informal
custom_stopwords = set(STOPWORDS)
custom_stopwords.update([
    'que', 'de', 'te', 'en', 'la', 'lo', 'el', 'las', 'los', 'por', 'es',
    'son', 'para', 'una', 'un', 'con', 'se', 'su', 'mi', 'tu', 'yo', 'al',
    'multimedia', 'omitido', 'https', 'www', 'com', 'jajaja', 'jaja', 'si', 'no', 'ya'
])

def limpiar_palabras(texto):
    texto = str(texto).lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto.split()

todas_las_palabras = []
for mensaje in df['Mensaje']:
    if '<Multimedia' not in mensaje:
        palabras = limpiar_palabras(mensaje)
        todas_las_palabras.extend(palabras)

texto_total = ' '.join(todas_las_palabras)

# Configuración del WordCloud (Sin máscara de corazón)
wordcloud = WordCloud(
    width=1000, height=500,
    background_color='black',
    stopwords=custom_stopwords,
    max_words=150,
    colormap='viridis', # Paleta de colores fría/tech
    # mask=np.array(Image.open('Resources/beer_icon.png')), # Descomentar si tienes el icono
).generate(texto_total)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("☁️ Palabras más usadas")
plt.show()